In [4]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import zipfile
import io
import pandas as pd
from IPython.display import display
from collections import defaultdict

zipped_data_path = "../data/clean_data/class-competition.zip"

dataframes = defaultdict(pd.DataFrame)
with zipfile.ZipFile(zipped_data_path, "r") as zip:
    for filename in zip.namelist():
        if filename.endswith(".csv"):
            with zip.open(filename) as f:
                dataframes.update({ filename : pd.read_csv(io.TextIOWrapper(f)) })

                # Lets take a look at the files
                print(f"FILE: {filename}")
                display(dataframes[filename].info())
                display(dataframes[filename].head())

train_data = dataframes["train.csv"]
test_data  = dataframes["test_public.csv"]

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

BadZipFile: Bad CRC-32 for file 'train.csv'

In [5]:
from sklearn.model_selection import train_test_split

ALL_FEATURES = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'MISSING_DATA',
 'POLYLINE', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C', 'YEAR_2013', 'YEAR_2014',
 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5', 'MONTH_6',
 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11', 'MONTH_12',
 'DAY_OF_WEEK_0', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3',
 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'HOUR_0', 'HOUR_1',
 'HOUR_2', 'HOUR_3', 'HOUR_4', 'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8',
 'HOUR_9', 'HOUR_10', 'HOUR_11', 'HOUR_12', 'HOUR_13', 'HOUR_14',
 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18', 'HOUR_19', 'HOUR_20',
 'HOUR_21', 'HOUR_22', 'HOUR_23']

# We could totally change this. Utilization of these just probably requires further preprocessing.
ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'POLYLINE']

train_data_sample = train_data.sample(frac=0.01, random_state=420) # frac is used to control percentage of train data used
X_sample = train_data_sample.drop("TRAVEL_TIME", axis=1)
X_sample = X_sample.loc[:, ~X_sample.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
y_sample = train_data_sample["TRAVEL_TIME"]
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=420)

NameError: name 'train_data' is not defined

In [6]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(49, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64, 1)
    )

  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)
   
mlp = MLP()
if torch.cuda.is_available():
    mlp.cuda()

In [7]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01)

In [8]:
for epoch in range(0, 10): # 10 epochs at maximum
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    X_train = torch.tensor(X_train.values)
    X_train = X_train.to(device)
    X_train = X_train.float()
    
    y_train = torch.tensor(y_train.values)
    y_train = y_train.to(device)
    y_train = y_train.float()
      
    # Zero the gradients
    optimizer.zero_grad()
    
    # Perform forward pass
    outputs = mlp(X_train)
    
    # Compute loss
    loss = criterion(outputs, y_train)
    
    # Perform backward pass
    loss.backward()
    
    # Perform optimization
    optimizer.step()

    current_loss += loss.item()
    print(current_loss)

# Process is complete.
print('Training process has finished.')

Starting epoch 1


NameError: name 'X_train' is not defined